In [1]:
!pip install ultralytics
!pip install roboflow
!pip install supervision

In [2]:
from google.colab import drive
from pathlib import Path
import os
import json
drive.mount('/content/drive')
x = '/content/drive/MyDrive/Football-Analysis-System/'
path = Path(x)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['clips',
 'yolov8x.pt',
 'runs',
 'training',
 'config.json',
 '.ipynb_checkpoints',
 'yolov8n.pt',
 'yolov5lu.pt',
 'yolo_inference.ipynb',
 'video_utils_trials.ipynb',
 'trackers',
 'stubs']

In [3]:
cd  /content/drive/MyDrive/Football-Analysis-System

/content/drive/MyDrive/Football-Analysis-System


In [4]:
from ultralytics import YOLO
import supervision as sv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [5]:
def read_video(video_path):
    """read_video : reads a video file and returns the frames

    Args:
        video_path : frames of the video
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    return frames


def save_video(output_video_frames, output_video_path):
    # XVID is the output format
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    # generate a video at the path, of type fourcc @ 24 frames/second and framesize as
    # the last parameter (width, height)
    out = cv2.VideoWriter(output_video_path, fourcc, 24,
                          (output_video_frames[0].shape[1], output_video_frames[0].shape[0]))
    for frame in output_video_frames:
        out.write(frame)
    out.release

In [1]:
class Tracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.tracker = sv.ByteTrack()

    def detect_frames(self, frames):
        batch_size = 20
        detections = []

        # Split frames into batches of size batch_size
        # and detect each batch separately
        # for faster processing and avoid memory issues

        for i in range(0, len(frames), batch_size):
            batch = frames[i:i+batch_size]
            detections_batch = self.model.predict(batch, conf=0.1)
            detections += detections_batch

        return detections

    def get_object_tracks(self, frames, read_from_stub=False, stub_path=None):

        if read_from_stub and stub_path is not None and os.path.exists(stub_path):
            with open(stub_path, 'rb') as f:
                tracks = pickle.load(f)
            return tracks

        detections = self.detect_frames(frames)
        tracks = {
            'players': [],
            'referees': [],
            'ball': []
        }

        for frame_num, detection in enumerate(detections):
            class_names = detection.names
            class_names_inv = {i: j for j, i in class_names.items()}

            # convert the detection to supervison format
            detection_sv = sv.Detections.from_ultralytics(detection)

            # convert goalkeeper to player object
            for index, class_id in enumerate(detection_sv.class_id):
                if class_names[class_id] == 'goalkeeper':
                    detection_sv.class_id[index] = class_names_inv['player']

            # Track objects
            detections_with_tracks = self.tracker.update_with_detections(
                detection_sv)

            tracks['players'].append({})
            tracks['referees'].append({})
            tracks['ball'].append({})

            # Tracking the players and the referee since they are more than one.
            # No need to track the ball using the detection with tracks since there is
            # only one ball in the video hence only one bounding ball
            for frame_detection in detections_with_tracks:
                bounding_box = frame_detection[0].tolist()
                class_id = frame_detection[3]
                track_id = frame_detection[4]

                if class_id == class_names_inv['player']:
                    tracks['players'][frame_num][track_id] = {
                        "bounding_box": bounding_box}
                elif class_id == class_names_inv['referee']:
                    tracks['referees'][frame_num][track_id] = {
                        "bounding_box": bounding_box}

            # Tracking the ball...
            for frame_detection in detection_sv:
                bounding_box = frame_detection[0].tolist()
                class_id = frame_detection[3]

                if class_id == class_names_inv['ball']:
                    # We're using a constant one since there's only one ball
                    tracks['ball'][frame_num][1] = {
                        "bounding_box": bounding_box}

            if stub_path is not None:
                with open(stub_path, 'wb') as f:
                    pickle.dump(tracks, f)

            return tracks

In [7]:
# Read video
video_frames = read_video('clips/input/08fd33_4.mp4')

# Initilize Tracker
tracker = Tracker('runs/models/best.pt')
# get object tracks
tracker.get_object_tracks(video_frames, read_from_stub=True, stub_path='stubs/track_stubs.pkl')

# save video
# Already saved, that's why it's commented out
# save_video(video_frames, 'clips/output/08fd33_4.avi')


0: 384x640 1 ball, 1 goalkeeper, 18 players, 3 referees, 1974.4ms
1: 384x640 1 ball, 1 goalkeeper, 18 players, 3 referees, 1974.4ms
2: 384x640 1 goalkeeper, 18 players, 3 referees, 1974.4ms
3: 384x640 1 ball, 1 goalkeeper, 20 players, 3 referees, 1974.4ms
4: 384x640 1 ball, 1 goalkeeper, 19 players, 3 referees, 1974.4ms
5: 384x640 1 goalkeeper, 19 players, 3 referees, 1974.4ms
6: 384x640 1 goalkeeper, 20 players, 3 referees, 1974.4ms
7: 384x640 1 ball, 1 goalkeeper, 21 players, 3 referees, 1974.4ms
8: 384x640 1 ball, 1 goalkeeper, 20 players, 3 referees, 1974.4ms
9: 384x640 1 ball, 1 goalkeeper, 20 players, 3 referees, 1974.4ms
10: 384x640 1 goalkeeper, 20 players, 3 referees, 1974.4ms
11: 384x640 21 players, 3 referees, 1974.4ms
12: 384x640 20 players, 3 referees, 1974.4ms
13: 384x640 22 players, 3 referees, 1974.4ms
14: 384x640 1 ball, 20 players, 3 referees, 1974.4ms
15: 384x640 1 ball, 21 players, 3 referees, 1974.4ms
16: 384x640 1 ball, 19 players, 3 referees, 1974.4ms
17: 384x64

{'players': [{1: {'bounding_box': [1309.2283935546875,
     447.515625,
     1350.951904296875,
     516.3836059570312]},
   2: {'bounding_box': [1369.71728515625,
     816.0930786132812,
     1443.90283203125,
     904.3945922851562]},
   3: {'bounding_box': [847.8286743164062,
     634.3111572265625,
     901.087646484375,
     721.0339965820312]},
   4: {'bounding_box': [872.7779541015625,
     361.761474609375,
     905.5018310546875,
     423.14501953125]},
   5: {'bounding_box': [1572.20068359375,
     611.7222290039062,
     1612.78564453125,
     695.373779296875]},
   6: {'bounding_box': [587.7855834960938,
     589.5963745117188,
     630.960205078125,
     671.918212890625]},
   7: {'bounding_box': [994.6576538085938,
     453.38214111328125,
     1027.41259765625,
     526.8286743164062]},
   8: {'bounding_box': [329.2890930175781,
     495.97467041015625,
     365.4928894042969,
     567.9395141601562]},
   9: {'bounding_box': [533.4428100585938,
     687.2322998046875,
  

In [4]:
import pickle
# Open the pickle file in binary read mode
with open('/home/nyangweso/Desktop/Projects/Football-Analysis-system/stubs/track_stubs_final.pkl', 'rb') as file:
    # Load the contents of the pickle file
    data = pickle.load(file)

# Display the contents
print(data)

{'players': [{1: {'bbox': [851.27490234375, 632.7650756835938, 900.3116455078125, 721.5734252929688]}, 3: {'bbox': [328.891357421875, 494.59674072265625, 364.2227478027344, 568.696044921875]}, 4: {'bbox': [1309.0023193359375, 446.935546875, 1349.9012451171875, 516.509033203125]}, 5: {'bbox': [587.555419921875, 589.6014404296875, 630.6329956054688, 672.4948120117188]}, 6: {'bbox': [1369.9041748046875, 815.928955078125, 1443.7584228515625, 903.4451293945312]}, 7: {'bbox': [994.9427490234375, 453.58837890625, 1027.4996337890625, 526.4969482421875]}, 8: {'bbox': [872.430908203125, 360.96954345703125, 905.631591796875, 423.65771484375]}, 9: {'bbox': [1229.984619140625, 430.86517333984375, 1262.6832275390625, 501.3833923339844]}, 10: {'bbox': [1276.1383056640625, 392.9593505859375, 1306.6666259765625, 462.97613525390625]}, 11: {'bbox': [359.6844482421875, 721.2495727539062, 395.7275695800781, 825.8789672851562]}, 12: {'bbox': [222.49412536621094, 513.4769287109375, 255.4835968017578, 594.000

In [3]:
import pickle
# Open the pickle file in binary read mode
with open('/home/nyangweso/Desktop/Projects/Football-Analysis-system/stubs/track_stubs.pkl', 'rb') as file:
    # Load the contents of the pickle file
    data = pickle.load(file)

# Display the contents
print(data)

{'players': [{24: {'bounding_box': [1309.2283935546875, 447.515625, 1350.951904296875, 516.3836059570312]}, 25: {'bounding_box': [1369.71728515625, 816.0930786132812, 1443.90283203125, 904.3945922851562]}, 26: {'bounding_box': [847.8286743164062, 634.3111572265625, 901.087646484375, 721.0339965820312]}, 27: {'bounding_box': [872.7779541015625, 361.761474609375, 905.5018310546875, 423.14501953125]}, 28: {'bounding_box': [1572.20068359375, 611.7222290039062, 1612.78564453125, 695.373779296875]}, 29: {'bounding_box': [587.7855834960938, 589.5963745117188, 630.960205078125, 671.918212890625]}, 30: {'bounding_box': [994.6576538085938, 453.38214111328125, 1027.41259765625, 526.8286743164062]}, 31: {'bounding_box': [329.2890930175781, 495.97467041015625, 365.4928894042969, 567.9395141601562]}, 32: {'bounding_box': [533.4428100585938, 687.2322998046875, 578.4244384765625, 784.4778442382812]}, 33: {'bounding_box': [374.4496765136719, 305.85552978515625, 400.8157653808594, 366.3091735839844]}, 3